# CMFGEN

Database from John Hillier’s CMFGEN, a radiative transfer code designed to solve the radiative transfer and statistical equilibrium equations in spherical geometry.

<div class="alert alert-info">

**Note:**
    
In this example, the data was downloaded from the [CMFGEN website](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) and extracted to the `/tmp/atomic` folder.
    
</div>



## Parsers

The CMFGEN parsers retrieves data from text files preserving its original form (no unit conversions) and stores it in DataFrames. Currently, just `osc`, `col` and `pho` files are supported.


### Levels, Lines and Collisions

Energy levels and spectral lines are stored in the `osc` files, while collisions strengths are provided by the `col` files.

In [ ]:
from carsus.io.cmfgen import CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser, CMFGENCollisionalStrengthsParser

In [ ]:
si2_lvl = CMFGENEnergyLevelsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

In [ ]:
si2_osc = CMFGENOscillatorStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

In [ ]:
si2_col = CMFGENCollisionalStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_col')

Header information is stored in the `header` attribute and the DataFrame in the `base` attribute:

In [ ]:
si2_col.header

In [ ]:
si2_lvl.base

### Photoionization Cross-sections

Photoionization cross-sections are provided by the `pho` files.

In [ ]:
from carsus.io.cmfgen import CMFGENPhoCrossSectionsParser

In [ ]:
si2_cross_sections = CMFGENPhoCrossSectionsParser('/tmp/atomic/SIL/II/16sep15/phot_nahar_A')

In [ ]:
si2_cross_sections.header

In this case, `base` is a list containing many DataFrames. Also, each individual DataFrame contains information relative to that specific target under the attribute `attr`.

In [ ]:
type(si2_cross_sections.base)

In [ ]:
len(si2_cross_sections.base)

In [ ]:
si2_cross_sections.base[0]

In [ ]:
si2_cross_sections.base[0].attrs

### Batch Convert Files to HDF5

To convert multiple CMFGEN files to the HDF5 format import the `hdf_dump` function.

<div class="alert alert-info">

**Note:**
    
This feature has been temporary disabled.
    
</div>

Required parameters are `cmfgen_dir`, `patterns` and `parser`, while `chunk_size` and `ignore_patterns` are optional.

## CMFGENReader

The `CMFGENReader` provides processed `levels`, `lines`, `ionization_energies` and `cross_sections` tables to work with.

In [ ]:
from carsus.io.cmfgen import CMFGENReader

In [ ]:
cmfgen_reader = CMFGENReader.from_config('Si 1; Si 2', '/tmp/atomic', ionization_energies=True, cross_sections=True)

In [ ]:
cmfgen_reader.levels

In [ ]:
cmfgen_reader.lines

In [ ]:
cmfgen_reader.cross_sections

In [ ]:
cmfgen_reader.ionization_energies